# Reading Data - Parquet Files

**Technical Accomplishments:**
- Introduce the Parquet file format.
- Read data from:
  - Parquet files without a schema.
  - Parquet files with a schema.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Getting Started

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
# Initialize Spark Session
spark = (SparkSession.builder
         .appName("Read CSV Data")
         .getOrCreate())

In [ ]:
print(spark)
spark

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Reading from Parquet Files

<strong style="font-size:larger">"</strong>Apache Parquet is a columnar storage format available to any project in the Hadoop ecosystem, regardless of the choice of data processing framework, data model or programming language.<strong style="font-size:larger">"</strong><br>

### About Parquet Files
* Free & Open Source.
* Increased query performance over row-based data stores.
* Provides efficient data compression.
* Designed for performance on large data sets.
* Supports limited schema evolution.
* Is a splittable "file format".

&nbsp;&nbsp;&nbsp;&nbsp;**Row Format** &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Column Format**

<table style="border:0">

  <tr>
    <th>ID</th><th>Name</th><th>Score</th>
    <th style="border-top:0;border-bottom:0">&nbsp;</th>
    <th>ID:</th><td>1</td><td>2</td>
    <td style="border-right: 1px solid #DDDDDD">3</td>
  </tr>

  <tr>
    <td>1</td><td>john</td><td>4.1</td>
    <td style="border-top:0;border-bottom:0">&nbsp;</td>
    <th>Name:</th><td>john</td><td>mike</td>
    <td style="border-right: 1px solid #DDDDDD">sally</td>
  </tr>

  <tr>
    <td>2</td><td>mike</td><td>3.5</td>
    <td style="border-top:0;border-bottom:0">&nbsp;</td>
    <th style="border-bottom: 1px solid #DDDDDD">Score:</th>
    <td style="border-bottom: 1px solid #DDDDDD">4.1</td>
    <td style="border-bottom: 1px solid #DDDDDD">3.5</td>
    <td style="border-bottom: 1px solid #DDDDDD; border-right: 1px solid #DDDDDD">6.4</td>
  </tr>

  <tr>
    <td style="border-bottom: 1px solid #DDDDDD">3</td>
    <td style="border-bottom: 1px solid #DDDDDD">sally</td>
    <td style="border-bottom: 1px solid #DDDDDD; border-right: 1px solid #DDDDDD">6.4</td>
  </tr>

</table>

See also
* <a href="https://parquet.apache.org/" target="_blank">https&#58;//parquet.apache.org</a>
* <a href="https://en.wikipedia.org/wiki/Apache_Parquet" target="_blank">https&#58;//en.wikipedia.org/wiki/Apache_Parquet</a>

### Data Source

The data for this example shows the number of requests to Wikipedia's mobile and desktop websites from Wikipedia. 

### Read in the Parquet Files

To read in this files, we will specify the location of the parquet directory.

In [ ]:
parquetFile = "../dataset/pageviews_by_second.parquet/"

(spark.read              # The DataFrameReader
  .parquet(parquetFile)  # Creates a DataFrame from Parquet after reading in the file
  .printSchema()         # Print the DataFrame's schema
)

### Review: Reading from Parquet Files
* We do not need to specify the schema - the column names and data types are stored in the parquet files.
* Only one job is required to **read** that schema from the parquet file's metadata.
* Unlike the CSV or JSON readers that have to load the entire file and then infer the schema, the parquet reader can "read" the schema very quickly because it's reading that schema from the metadata.

### Read in the Parquet Files w/Schema

If you want to avoid the extra job entirely, we can, again, specify the schema even for parquet files:

** *WARNING* ** *Providing a schema may avoid this one-time hit to determine the `DataFrame's` schema.*  
*However, if you specify the wrong schema it will conflict with the true schema and will result in an analysis exception at runtime.*

In [ ]:
# Required for StructField, StringType, IntegerType, etc.
from pyspark.sql.types import *

parquetSchema = StructType(
  [
    StructField("timestamp", StringType(), False),
    StructField("site", StringType(), False),
    StructField("requests", IntegerType(), False)
  ]
)

(spark.read               # The DataFrameReader
  .schema(parquetSchema)  # Use the specified schema
  .parquet(parquetFile)   # Creates a DataFrame from Parquet after reading in the file
  .printSchema()          # Print the DataFrame's schema
)

Let's take a look at some of the other details of the `DataFrame` we just created for comparison sake.

In [ ]:
parquetDF = spark.read.schema(parquetSchema).parquet(parquetFile)

print("Partitions: " + str(parquetDF.rdd.getNumPartitions()) )


In most/many cases, people do not provide the schema for Parquet files because reading in the schema is such a cheap process.

And lastly, let's peek at the data:

In [ ]:
parquetDF.show(10)

In [ ]:
spark.stop()